In [1]:
import pandas as pd

Currently this file is the same as dataPreprocessing_and_queries (as of 1/25/23) 
My intention is to make this file only have the path to taking csvs into the dataframe, so it's all in one place without any extras. 

In [2]:

#make a dictionary storying the year, file name, and path names.
amz_info_dict = {}
years = [2019, 2020, 2021]

root_path = '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/'

for year in years:
    year_dict={}

    year_dict['year'] = year

    file_name = "amz_" + str(year) + '.csv'
    year_dict['file_name'] = file_name

    path = root_path + file_name
    year_dict['path'] = path

    year_dict['data_included'] = False #this is a way for us to keep track of what data we have included in the main dataframe or not.

    amz_info_dict[year] = year_dict
print(amz_info_dict)

{2019: {'year': 2019, 'file_name': 'amz_2019.csv', 'path': '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/amz_2019.csv', 'data_included': False}, 2020: {'year': 2020, 'file_name': 'amz_2020.csv', 'path': '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/amz_2020.csv', 'data_included': False}, 2021: {'year': 2021, 'file_name': 'amz_2021.csv', 'path': '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/amz_2021.csv', 'data_included': False}}


In [3]:
#this is how we read the csv out of the loop, for reference:
#amz_2021 = pd.read_csv('/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/amz_2021.csv')

In [4]:
#read csv for each year and store it in the name 'amz_year' 
for year_dict in amz_info_dict.values():
    #print(year_dict)
    var_file_name = year_dict['file_name']
    var_path = year_dict['path']
    var_file_name = pd.read_csv(var_path, header= [0, 1])
    year_dict['csv'] = var_file_name


#amz_info_dict[2021]['csv']
#the above is how you access the csv for each year
#now lets get it into a dataframe. 


In [73]:
amz_2021 = pd.DataFrame(data = amz_info_dict[2021]['csv'])

#here's all the lists that we will need to set up the headers for the data.
#this gets the list of job categories, now NOT including totals. 
job_categories = amz_2021['Gender']['Race']

job_categories = list(job_categories)
job_categories = job_categories[:-2]
job_categories



['Exec/Sr. Officials & Mgrs',
 'First/Mid Officials & Mgrs',
 'Professionals',
 'Technicians',
 'Sales Workers',
 'Administrative Support',
 'Craft Workers',
 'Operatives',
 'Laborers & Helpers',
 'Service Workers']

In [6]:
#let's get the list of headers that we want
EEO1_col_names = ['count_employees', 'job_category', 'gender', 'race', 'year', 'company']
#and set up the dataframe all this will be going into.
EEO1_data_long = pd.DataFrame(columns = EEO1_col_names)

EEO1_data_long

,count_employees,job_category,gender,race,year,company


In [7]:
#drop the column that is job category.
#also drop the last column that is the totals.
amz_2021 = amz_2021.drop(['Gender', 'Unnamed: 15_level_0'], axis=1)

#also cut off the bottom two rows.
amz_2021 = amz_2021.iloc[:-2 , :]

#amz_2021


/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/716787239.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  amz_2021 = amz_2021.drop(['Gender', 'Unnamed: 15_level_0'], axis=1)


In [77]:
#this stands for Gender Race Columns.  It is tuples of all the combinations we have for male and female and races.
GRcols = amz_2021.columns.values
GRcols


array([('Gender', 'Race'), ('Male', 'Hispanic or Latino'),
       ('Male', 'White'), ('Male', 'Black or African American'),
       ('Male', 'Native Hawaiian Or Pacific\nIslander'),
       ('Male', 'Asian'), ('Male', 'American Indian or Alaska\nNative'),
       ('Male', 'Two or More Races'), ('Female', 'Hispanic or Latino'),
       ('Female', 'White'), ('Female', 'Black or African American'),
       ('Female', 'Native Hawaiian Or Pacific\nIslander'),
       ('Female', 'Asian'),
       ('Female', 'American Indian or Alaska\nNative'),
       ('Female', 'Two or More Races'),
       ('Unnamed: 15_level_0', 'Unnamed: 15_level_1')], dtype=object)

In [80]:
#Front end asked for list of races as they come here. Going to make that here.
lst_of_races = []
for elem in GRcols[1:-1]:
    lst_of_races.append(elem[1])

lst_of_races

['Hispanic or Latino',
 'White',
 'Black or African American',
 'Native Hawaiian Or Pacific\nIslander',
 'Asian',
 'American Indian or Alaska\nNative',
 'Two or More Races',
 'Hispanic or Latino',
 'White',
 'Black or African American',
 'Native Hawaiian Or Pacific\nIslander',
 'Asian',
 'American Indian or Alaska\nNative',
 'Two or More Races']

In [9]:
"""
#this is how we do it for one column.
col1 = amz_2021['Male']['Hispanic or Latino']
#recall job_categories is the list of job_categories.
year = 2021
company = 'Amazon'
gender = 'Male'
race= 'Hispanic or Latino'
for i in range(len(col1)):
    row_lst=[col1[i], job_categories[i], gender, race, year, company]
    row_dict = dict(zip(EEO1_col_names, row_lst))
    #row_ser=pd.Series(row_lst)
    EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)
"""

"\n#this is how we do it for one column.\ncol1 = amz_2021['Male']['Hispanic or Latino']\n#recall job_categories is the list of job_categories.\nyear = 2021\ncompany = 'Amazon'\ngender = 'Male'\nrace= 'Hispanic or Latino'\nfor i in range(len(col1)):\n    row_lst=[col1[i], job_categories[i], gender, race, year, company]\n    row_dict = dict(zip(EEO1_col_names, row_lst))\n    #row_ser=pd.Series(row_lst)\n    EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)\n"

In [10]:
#iterate over GR cols to get all the information we want.
#this puts it in line by line to our final dataframe.
for gender, race in GRcols:
    col = amz_2021[gender][race]
    year= 2021
    company = 'Amazon'
    for i in range(len(col)):
        row_lst=[col[i], job_categories[i], gender, race, year, company]
        row_dict = dict(zip(EEO1_col_names, row_lst))
        #row_ser=pd.Series(row_lst)
        EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)

#update the dictionary to say that the data in included, and include the final dataframe (that is all clean) in the amz_info_dict. 
#that way everything is in one place, the info dict.
amz_info_dict[2021]['data_included'] = True
amz_info_dict[2021]['data_frame'] = amz_2021
    

/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/1349923645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)
/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/1349923645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)
/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/1349923645.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprec

In [11]:
EEO1_data_long.head(20)

,count_employees,job_category,gender,race,year,company
0,106,Exec/Sr. Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
1,5192,First/Mid Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
2,4785,Professionals,Male,Hispanic or Latino,2021,Amazon
3,1240,Technicians,Male,Hispanic or Latino,2021,Amazon
4,9819,Sales Workers,Male,Hispanic or Latino,2021,Amazon
5,1577,Administrative Support,Male,Hispanic or Latino,2021,Amazon
6,224,Craft Workers,Male,Hispanic or Latino,2021,Amazon
7,7998,Operatives,Male,Hispanic or Latino,2021,Amazon
8,95484,Laborers & Helpers,Male,Hispanic or Latino,2021,Amazon
9,1653,Service Workers,Male,Hispanic or Latino,2021,Amazon


## Summmary of work January 23, 2023

I have made a dataframe with the long data for amazon 2021.  This is the form we want it in.
Next steps: revise this to also include the other files. (ideally, loop through this process in a for loop.  or, make helper functions to take care of this.)
Make it so that the EEO1_data_long has all three years of amazon that we have.

I think we should take out the 'total' columns. (these are at the end of the GR columns.)  Also I should get rid of the total columns in the job category title list.  Because we should be able to calculate these on our own.

However, I need to check with margaret about this.  These values could be useful to check our work; however, I think they should just be removed at this point.

After we get the data in long form from amazon, I would like to try doing some queries on this.

We should set up the data in a database and get that deployed.
I may also have to research what is the best way to do queries.

I'd also like to see what graphs I can make on the backend.

Question for Auberon: how can we use .concat instead of .append, because it's throwing warnings. (this is low priority.)





## Work January 24, 2023

I took out the total column on the far right and the total row on the bottom. 
We may have a use for these later, but it makes the most sense to get rid of them now and calculate them ourselves, I believe.
(To do: confirm that the totals are adding up the other numbers; that is, no employee is counted twice in the EEO-1)

Now, let's iterate over all the years so that we get the data from the other two Amazon years into this data frame. 

In [12]:
#amz_info_dict

for year, year_dict in amz_info_dict.items():
    if year_dict['data_included'] == False:
        #I'll just call it df for now.  then I'll save it in the amz_info_dict, so that's how we can access it. 
        df = pd.DataFrame(data = amz_info_dict[year]['csv'])
        #drop the column that is job category.
        #also drop the last column that is the totals.
        df = df.drop(['Gender', 'Unnamed: 15_level_0'], axis=1)
        #also cut off the bottom two rows.
        df = df.iloc[:-2 , :]

        #iterate over GR cols to get all the information we want.
        #this puts it in line by line to our final dataframe.
        for gender, race in GRcols:
            col = df[gender][race]
            #year = year; year is already defined in this loop over the dict items.
            company = 'Amazon'
            for i in range(len(col)):
                row_lst=[col[i], job_categories[i], gender, race, year, company]
                row_dict = dict(zip(EEO1_col_names, row_lst))
                #row_ser=pd.Series(row_lst)
                EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)

        #update the dictionary to say that the data is included, and include the final dataframe (that is all clean) in the amz_info_dict. 
        #that way everything is in one place, the info dict.
        amz_info_dict[year]['data_included'] = True
        amz_info_dict[year]['data_frame'] = df  #this is where the data frame is saved.

/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/3580250919.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(['Gender', 'Unnamed: 15_level_0'], axis=1)
/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/3580250919.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)
/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784/3580250919.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  EEO1_data_long = EEO1_data_long.append(row_dict, ignore_index=True)  #this works but it's deprecated. (ask for help on this)
/var/folders/q2/g4rvv9lj4594r_pxcwzdhr6h0000gn/T/ipykernel_33784

In [14]:
EEO1_data_long

,count_employees,job_category,gender,race,year,company
0,106,Exec/Sr. Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
1,5192,First/Mid Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
2,4785,Professionals,Male,Hispanic or Latino,2021,Amazon
3,1240,Technicians,Male,Hispanic or Latino,2021,Amazon
4,9819,Sales Workers,Male,Hispanic or Latino,2021,Amazon
...,...,...,...,...,...,...
415,699,Administrative Support,Female,Two or More Races,2020,Amazon
416,2,Craft Workers,Female,Two or More Races,2020,Amazon
417,652,Operatives,Female,Two or More Races,2020,Amazon
418,11157,Laborers & Helpers,Female,Two or More Races,2020,Amazon


In [ ]:
#note: this creates a file with an index which ends up not being needed.  I should do drop index = true?

In [17]:
EEO1_data_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   count_employees  420 non-null    object
 1   job_category     420 non-null    object
 2   gender           420 non-null    object
 3   race             420 non-null    object
 4   year             420 non-null    object
 5   company          420 non-null    object
dtypes: object(6)
memory usage: 19.8+ KB


In [19]:
#get the data to the correct data types.
convert_dict = {'count_employees': 'int64',
                'job_category': 'category',
                'gender': 'category',
                'race': 'category',
                'year': 'int64',
                'company': 'category'}

EEO1_data_long = EEO1_data_long.astype(convert_dict)

print(EEO1_data_long.dtypes)

count_employees       int64
job_category       category
gender             category
race               category
year                  int64
company            category
dtype: object


In [20]:
#now let's save this dataframe as a CSV so I can share it with my team.

EEO1_data_long.to_csv('/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/amz_long_csv.csv')

### Getting this on your local machine

- Open postgres with psql -U postgres
- CREATE DATABASE eeo1_db;
- run the code below (uncomment out the pip install the first time)

In [29]:
#pip install psycopg2

In [30]:
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://postgres@localhost:5432/eeo1_db")

In [31]:
#now let's take the dataframe and put it on my postgres database locally.

EEO1_data_long.to_sql('eeo1_table', con = engine)

420

In [70]:
#this will only display the last query.  If you want to see them all, simply group these into different code windows. 
#now we can do sql queries in here using engine:
#simple queries:
#fetch all:
all_data = engine.execute("SELECT * FROM eeo1_table").fetchall()

#get total by gender:
total_gender = engine.execute("SELECT gender, SUM(count_employees) FROM eeo1_table GROUP BY gender").fetchall()

#get total by race:
total_race = engine.execute("SELECT race, SUM(count_employees) FROM eeo1_table GROUP BY race").fetchall()

#get overall job breakdown.
total_job_category = engine.execute("select job_category, sum(count_employees) from eeo1_table group by job_category").fetchall()

In [72]:
total_race
#this gives me lists of tuples, with labels and with the values.
#front end wants list of labels, and list of numbers.
#in a json with labels : ['race', 'race'] and data: [num, num, num]

[('American Indian or Alaska\nNative', Decimal('22419')),
 ('Two or More Races', Decimal('88492')),
 ('Asian', Decimal('337054')),
 ('White', Decimal('876428')),
 ('Black or African American', Decimal('667642')),
 ('Native Hawaiian Or Pacific\nIslander', Decimal('14873')),
 ('Hispanic or Latino', Decimal('571510'))]

In [40]:
#queries with multiple fields:
job_category_by_gender = engine.execute("SELECT job_category, gender, sum(count_employees) FROM eeo1_table GROUP BY job_category, gender ORDER BY job_category, gender").fetchall()


In [45]:
#make the list of lists a dataframe so that we can then use the .to_csv method.
#I'm going to want to make a helper function that takes in the group_by parameters and gets this csv to export.  
job_category_by_gender = pd.DataFrame(job_category_by_gender, columns = ['job_category', 'gender', 'count_employees'])
job_category_by_gender.to_csv('/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/query_results/job_category_by_gender.csv')

In [46]:
job_category_by_race = engine.execute("SELECT job_category, race, sum(count_employees) FROM eeo1_table GROUP BY job_category, race ORDER BY job_category, race").fetchall()

In [47]:
job_category_by_race = pd.DataFrame(job_category_by_race, columns = ["job_category", "race", "count_employees"])
job_category_by_race.to_csv('/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/query_results/job_category_by_race.csv')

In [66]:
#Okay, lets make a helper function to get these queries into csvs and save them.
#given the results of a query, stored as a variable (and a list of lists), and a list of the columns it needs, 
#convert it to a dataframe with the correct columns.
#convert it to a csv, saved in the path file with the correct name.


def query_to_saved_csv(query_result, cols, query_root_path, file_name):  #is there a way to get the file name from the name of the query_result variable?
    #note that query_root_path isn't working...
    """given the results of a query, stored as a variable (and a list of lists), 
    and a list of the column names it needs, and the folder path, 
    save it as a csv, saved in the path file with the correct name.
    Note that there's no return, just the csv gets saved."""
    query_df = pd.DataFrame(query_result, columns = cols)
    print(query_df)
    #query_df.to_csv(root_path + file_name + '.csv')  #THIS DIDN'T WORK; NOT SURE WHY.  PUTTING IT IN MANUALLY FOR NOW. 
    query_df.to_csv('/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/query_results/' + file_name + '.csv')
    

query_root_path = '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/amazon/tableOnlyCsvs/query_results/'



In [67]:
query_to_saved_csv(total_gender, ['gender', 'count_employees'], query_root_path, 'total_gender')

   gender count_employees
0  Female         1191208
1    Male         1387210


In [68]:
query_to_saved_csv(total_race, ['race', 'count_employees'], query_root_path, 'total_race')

                                   race count_employees
0     American Indian or Alaska\nNative           22419
1                     Two or More Races           88492
2                                 Asian          337054
3                                 White          876428
4             Black or African American          667642
5  Native Hawaiian Or Pacific\nIslander           14873
6                    Hispanic or Latino          571510


In [69]:
query_to_saved_csv(total_job_category, ['job_category', 'count_employees'], query_root_path, 'total_job_category')

                 job_category count_employees
0               Sales Workers          215031
1                 Technicians           21695
2  First/Mid Officials & Mgrs          135737
3             Service Workers           23543
4               Professionals          300154
5   Exec/Sr. Officials & Mgrs            7798
6          Laborers & Helpers         1694667
7               Craft Workers            2868
8                  Operatives          116746
9      Administrative Support           60179
